In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
import torch

# Configuração do modelo Phi-3-1.3B com quantização 4-bit
model_id = "microsoft/Phi-3-mini-4k-instruct"  # Substitua pelo ID exato do modelo Phi-3-1.3B no Hugging Face Hub

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Carregar o modelo e tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Criar pipeline para geração de texto
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)

# Integração com LangChain
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_28493/4081515703.py:32: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [2]:
# PHI 3
template_rag = """
<|system|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais. sempre em portugues do brasil, respondendo de forma clara e objetiva<|end|>
<|user|>
Pergunta: {pergunta}
---
Contexto: {contexto}
<|end|>
<|assistant|>
"""

In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader(web_paths = ("https://www.kalangosmotoclube.com.br/quem-somos.html",),)
docs = loader.load()

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, add_start_index = True)
splits = text_splitter.split_documents(docs)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

/tmp/ipykernel_28493/2278812390.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")


In [7]:
from langchain_chroma import Chroma
#vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)
# Crie o vectorstore com persistência local
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=hf_embeddings,
    persist_directory="./data/chroma"
)

# caso fossemos usar os embeddings da Open AI, basta mudar o método, passando direto conforme abaixo
#vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [8]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})

In [9]:
from langchain.prompts import PromptTemplate

prompt_rag = PromptTemplate(
    input_variables=["contexto", "pergunta"],
    template=template_rag,
)
prompt_rag

PromptTemplate(input_variables=['contexto', 'pergunta'], input_types={}, partial_variables={}, template='\n<|system|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais. sempre em portugues do brasil, respondendo de forma clara e objetiva<|end|>\n<|user|>\nPergunta: {pergunta}\n---\nContexto: {contexto}\n<|end|>\n<|assistant|>\n')

In [10]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain_rag = ({"contexto": retriever | format_docs, "pergunta": RunnablePassthrough()}
             | prompt_rag
             | llm
             | StrOutputParser())

In [12]:
chain_rag.invoke("Quando foi fundado o Kalangos Motoclube?")

' O Kalangos Motorcycle Club (KMC) teve sua data oficial de fundação documentada pela primeira vez na edição número 146 do jornal "Duas Rodas", onde se destacou como o segundo melhor motorclube brasileiro após a vitória nas competições organizadas pelas Associações de Corredores Amadores - ACAs. Apesar disso não estar explicitamente declarado como a data exata de fundação, podemos inferir que deve ter sido estabelecido antes dessa conquista notável mencionada. As informações fornecidas nos artigos referentes aos direitos e deveres dos membros indicam práticas sólidas dentro da entidade, enfatizando valores como responsabilidade, participação democrática e contribuição positiva à sociedade local. Além disso, o KMC possui reconhecimento nacional devido à presença constante em diversas revistas especializadas sobre motociclismo, refletindo seu envolvimento contínuo e crescente tanto em aspectos recreativos quanto profissionais.'

In [15]:
chain_rag.invoke("o que Caberá ao Diretor de Disciplina?")

' As funções descritas nos artigos referenciados são essenciais para manter uma estrutura organizacional funcional dentro de instituições como clubes esportivos ou sociedades similares. Cada cargo tem seu papel específico na gestão geral dos negócios diários e políticos do grupo. Abaixo segue uma explicação detalhada dessas funções conforme elas foram delineadas nas letras correspondentes às respectivas normativas:\n\n**Função do Diretor de Disciplina (Artigo 25):**  \nEste indivíduo é encarregado de garantir a integridade moral e disciplinar entre os membros do clube. Sua principal tarefa é notificar tanto os conselhos quanto o presidente sobre quaisquer falhas éticas ou comportamentos inadequados por parte dos sócios. Após receber informações sobre tal situação, este oficial fará valer seus poderes discursivos durante sessões conjuntas - conhecidas como assembleias - onde poderá exercer audiências justas e imparciais, permitindo assim que ambos lados tenham voz em questões relacionad

In [14]:
docs

[Document(metadata={'source': 'https://www.kalangosmotoclube.com.br/quem-somos.html', 'title': 'Kalangos Moto Clube | Nossa história', 'description': 'Conheça a história do Kalangos Moto Clube.', 'language': 'pt-br'}, page_content="\n\n\n\n\n\n\n\nKalangos Moto Clube | Nossa história\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBarra de Navegação\nMENU\n\n\n\n\n\nHome\n\nQuem Somos\n\n\nIntegrantes\n\nMídia\nContato\n\n\n\n\n\n\n\nO Moto Clube\nConheça a nossa história\n\n\n\n          O Kalango’s Moto Clube e uma entidade sem fins lucrativos, devidamente\n          registrado no 2º Cartório de registros de Titulos, que visa a promoção\n          da fraternidade e a cooperação entre os motociclistas, defendendo no\n          âmbito de suas atribuições a boa imagem do motociclista, participando\n          sempre de atividades no meio motociclistico. Fundado em 25 de janeiro\n          de 1995, na Zona Sul de São Paulo, trazendo como principal proposta a\n          união dentro do motociclismo e em todo

In [16]:
chain_rag.invoke("Quai são os requisitos para admissão?")

' Artigo 12 - Punições\nAs penalidades decorrentes da infração dos princípios estabelecidos pelo Moto Clube podem variar conforme a gravidade do caso e ser decididas pelas assembleias competentes. Em geral, estes incluem advertências, onde se solicita à pessoa incriminada uma reflexão sobre seus comportamentos; desligamento temporário ou definitivo, dependendo da severidade da transgressão, exigindo também a devolução do escudo associativo como simbolismo de renúncia aos privilégios do clube até que possa provar reforma moral adequada.\n\nOs artigos referentes às assembléias mencionam explicitamente quais tipos de questões elas tratam e quantas vezes anualmente acontecem estas reuniões especificamente designadas para tarefas distintas entre si. Por exemplo, a Assembléia Ordinária tem lugar semanalmente nos domingos, enquanto a Extraordinária pode ser chamada a qualquer momento para lidar com situações imediatas, como mudanças nas lideranças ou outras emergencias necessitando tomada de 